# ML.Net - BookRecommendations

In [2]:
// ML.NET Nuget packages installation
#r "nuget:Microsoft.ML"

Installed package Microsoft.ML version 1.5.0

In [4]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;
using System;
using System.Collections.Generic;
using System.IO;
using System.Linq;

In [5]:
public class Book
{
    public int BookId { get; set; }
    public string BookTitle { get; set; }
    public string Author { get; set; }
    public string Genre1 { get; set; }
    public string Genre2 { get; set; }
}

public class BookRating
{
    public float Label;

    public float user;

    public float bookid;
}

public class BookRatingPrediction
{
    public float Label;

    public float Score;
}

In [6]:
public static string dataLocation = "./Data";
public static int bookPredictionId = 34941133;

In [7]:
private static IList<Book> LoadBookData()
{
    var result = new List<Book>();

    var reader = File.OpenRead($"{dataLocation}/bookfeatures.csv");

    var isHeader = true;
    var line = String.Empty;

    using (var streamReader = new StreamReader(reader))
    {
        while(!streamReader.EndOfStream)
        {
            if (isHeader)
            {
                line = streamReader.ReadLine();
                isHeader = false;
            }

            line = streamReader.ReadLine();
            var data = line.Split(',');

            var book = new Book
            {
                BookId = int.Parse(data[3].ToString()),
                BookTitle = data[8].ToString(),
                Author = data[1].ToString(),
                Genre1 = data[4].ToString(),
                Genre2 = data[5].ToString()
            };

            result.Add(book);
        }
    }

    return result;
}

In [8]:
var trainingDataPath = $".datasets/books/ratings.csv";

var context = new MLContext();

var reader = context.Data.TextReader(new TextLoader.Arguments() {
    Separator = ",",
    HasHeader = true,
    Column = new[]
    {
        new TextLoader.Column("Label", DataKind.R4, 0),
        new TextLoader.Column("user", DataKind.R4, 1),
        new TextLoader.Column("bookid", DataKind.R4, 2),
    }
});

IDataView data = reader.Read(trainingDataPath);

var (trainData, testData) = context.BinaryClassification.TrainTestSplit(data, testFraction: 0.2);

var pipeline = context.Transforms.Categorical.MapValueToKey("user", "userIdEncoded")
    .Append(context.Transforms.Categorical.MapValueToKey("bookid", "bookIdEncoded"))
    .Append(new MatrixFactorizationTrainer(context, "Label", "userIdEncoded", "bookIdEncoded", 
        advancedSettings: s => { s.NumIterations = 20; s.K = 100; }));

Console.WriteLine("Training recommender" + Environment.NewLine);
var model = pipeline.Fit(trainData);

var prediction = model.Transform(testData);
var metrics = context.Regression.Evaluate(prediction);

Console.WriteLine($"Model metrics: RMS - {metrics.Rms} R^2 - {metrics.RSquared}" + Environment.NewLine);

var predictionFunc = model.MakePredictionFunction<BookRating, BookRatingPrediction>(context);

var bookPrediction = predictionFunc.Predict(new BookRating {
    user = 99,
    bookid = bookPredictionId
});

var bookData = LoadBookData();

Console.WriteLine($"Predicted rating - {Math.Round(bookPrediction.Score, 1)} for book {bookData.FirstOrDefault(b => b.BookId == bookPredictionId).BookTitle}");

Console.ReadLine();

Unhandled exception: (5,53): error CS0426: O nome de tipo "Arguments" não existe no tipo "TextLoader"
(10,49): error CS0117: '"DataKind" não contém uma definição para "R4"
(11,48): error CS0117: '"DataKind" não contém uma definição para "R4"
(12,50): error CS0117: '"DataKind" não contém uma definição para "R4"
(5,27): error CS1061: ‘DataOperationsCatalog’ não contém uma definição para "TextReader" e não foi possível encontrar nenhum método de extensão "TextReader" que aceite um primeiro argumento do tipo ‘DataOperationsCatalog’ (você está se esquecendo de usar uma diretiva ou uma referência de assembly?)
(18,6): error CS8130: Não é possível inferir o tipo da variável de desconstrução digitada implicitamente 'trainData'.
(18,17): error CS8130: Não é possível inferir o tipo da variável de desconstrução digitada implicitamente 'testData'.
(20,16): error CS1929: '"TransformsCatalog.CategoricalTransforms" não contém uma definição para "MapValueToKey" e a melhor sobrecarga do método d